In [1]:
using Gridap
using GridapGmsh
using Gridap.FESpaces
using Gridap.Geometry
using PyCall
using PyPlot
using JLD
using StatsBase

In [2]:
function solucion_en_malla(Ω,uh)
    ndcel=Ω.grid.cell_node_ids
    ndcoor=Ω.grid.node_coordinates
    u=zeros(length(ndcel))
    sols=uh.cell_dof_values
    for i in 1:length(ndcel)
        for j in 1:length(sols[i])
            u[ndcel[i][j]]=sols[i][j]
        end
    end    
    return ndcoor,u
end

function pasar_a_cartesianas(ndcoor,u,dx,dy)
    x=Int[]
    y=Int[]
    for nod in ndcoor
        append!(x,Int(round(nod[1]/dx))+1)
        append!(y,Int(round(nod[2]/dy))+1)
    end
    matA=zeros(maximum(x),maximum(y))
    for i in 1:length(ndcoor)
        matA[x[i],y[i]]=u[i]
    end
    return matA
end

pasar_a_cartesianas (generic function with 1 method)

In [3]:
function resolver_una_etapa_decision(Pec,T_1,T_2,S,vaf,V0,Ω,dΩ,Γ,dΓ,dx,dy,dt,t0,Tf,u0,com)
    g1(x,t::Real)=T_1
    g1(t::Real) = x -> g1(x,t)
    g2(x,t::Real)=T_2
    g2(t::Real) = x -> g2(x,t)
    Ug = TransientTrialFESpace(V0,[g1,g2])

    m(u,v) = ∫( u*v )dΩ
    a(u,v) = (1.0/Pec)*∫( (∇(u)⋅∇(v)) )dΩ +∫( (vaf⋅(∇(u)))*v)dΩ
    b(t,v) = ∫( S(t)*v )dΩ
    op_Af = TransientConstantMatrixFEOperator(m,a,b,Ug,V0)    
    linear_solver = LUSolver()
    Δt = dt
    θ = 0.5
    ode_solver = ThetaMethod(linear_solver,Δt,θ)
    t₀ = t0
    T = Tf
    u₀=interpolate_everywhere(0.0,Ug(0.0))
    if !com
        u₀=u0
    end
    uₕₜ= solve(ode_solver,op_Af,u₀,t₀,T)
    tiempos=Float64[]
    soluciones=Array{Float64}[]
    ndcoor,u=solucion_en_malla(Ω,u₀)
    matA=pasar_a_cartesianas(ndcoor,u,dx,dy)
    append!(soluciones,[matA])
    append!(tiempos,t₀)
    uret=u₀
    tret=0.0
    for (uₕ,t) in uₕₜ
        #ndcoor,u=solucion_en_malla(Ω,uₕ)
        #matA=pasar_a_cartesianas(ndcoor,u,dx,dy)
        #append!(soluciones,[matA])
        #append!(tiempos,t)
        uret=uₕ
        tret=t
    end    
    ndcoor,u=solucion_en_malla(Ω,uret)
    matA=pasar_a_cartesianas(ndcoor,u,dx,dy)
    return tret,[matA],uret
end

resolver_una_etapa_decision (generic function with 1 method)

In [4]:
model = GmshDiscreteModel("geometria.msh")
#writevtk(model,"model")
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
V0 = TestFESpace(model,reffe;conformity=:H1,dirichlet_tags=["Dv1","Dv2"])
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)
neumanntags = ["Neumman"]
Γ = BoundaryTriangulation(model,tags=neumanntags)
dΓ = Measure(Γ,degree)

Info    : Reading 'geometria.msh'...
Info    : 11 entities
Info    : 2601 nodes
Info    : 5200 elements
Info    : Done reading 'geometria.msh'


Measure()

In [5]:
function S(x,t,posc)
    c=posc(t)
    if abs(c[1]-x[1])<0.1 && abs(c[2]-x[2])<0.1
        return 200.0
    else
        return 0.0
    end
end

function T1_aux(a)
    if a=="OO" || a=="OF"
        return -0.5
    else
        return 0.0
    end
end

function T2_aux(a)
    if a=="OO" || a=="FO"
        return -0.5
    else
        return 0.0
    end
end

function vax(a,x)
    if a=="OO"
        return VectorValue(0.0,5.0)
    elseif a=="OF"
        if x[1]<0.5
            return VectorValue(0.0,5.0)
        else
            return VectorValue(0.0,0.0)
        end
    elseif a=="FO"
        if x[1]>0.5
            return VectorValue(0.0,5.0)
        else
            return VectorValue(0.0,0.0)
        end
    else
        return VectorValue(0.0,0.0)
    end
end

vax (generic function with 1 method)

In [6]:
np = pyimport("numpy")
function simular_camino(Pec,V0,Ω,dΩ,Γ,dΓ,dx,dy,Ndeci,Tf,fdeci)
    t2=LinRange(0.0, Tf, Ndeci+1)
    pol=["FF"]
    x0=rand()
    y0=0.4+0.6*rand()
    posc(t::Real)=(x0-0.1*t,y0-0.1*t)
    St(t::Real)= x -> S(x,t,posc)
    ttodos=Float64[]
    solTodos=Array{Float64,2}[]
    com=true
    u0=Gridap.FESpaces.FEFunction
    for i in 1:Ndeci  
        T1f=T1_aux(pol[i])
        T2f=T2_aux(pol[i])
        vg(x)=vax(pol[i],x) 
        temps,soluciones,usig=resolver_una_etapa_decision(Pec,T1f,T2f,St,vg,V0,Ω,dΩ,Γ,dΓ,dx,dy,0.05,t2[i],t2[i+1],u0,com)
        append!(ttodos,temps)
        append!(solTodos,soluciones)
        u0=usig
        anew=fdeci(soluciones[end])
        append!(pol,[anew])
        com=false
    end
    save("data/solpruba.jld", "sol",solTodos, "t", ttodos, "centroS",posc.(ttodos),"tiemposDeci",t2,"pol",pol)
    return ttodos,solTodos
end

simular_camino (generic function with 1 method)

In [7]:
function f_random(x)
    return StatsBase.sample(["OO","OF","FO","FF"])
end

f_random (generic function with 1 method)

In [8]:
@time ttodos,solTodos=simular_camino(1.0,V0,Ω,dΩ,Γ,dΓ,1.0/50.0,1.0/50.0,20,10.0,f_random);

 24.007652 seconds (87.81 M allocations: 5.954 GiB, 5.88% gc time, 86.73% compilation time)


In [9]:
solTodos

20-element Vector{Matrix{Float64}}:
 [0.05244754022828531 0.10489753359205216 … 3.267362180237119 3.2616670808135675; 0.0 0.09293921987661038 … 3.2630884827633047 3.2576128317696518; … ; 0.0 0.04577506084660227 … 1.3603826618705197 1.361074579129067; 0.025790913889335285 0.051592231837095645 … 1.3593956682915427 1.3601266246947101]
 [-0.14276199534979128 -0.26070859607022767 … 0.3218491403375341 0.3206912854215771; -0.5 -0.2272008028023459 … 0.31997140232597965 0.3187323944417354; … ; -0.5 -0.24033696052792264 … -0.37125048512993614 -0.37100568645266024; -0.1671776850140072 -0.28022139475709684 … -0.3714988808238653 -0.3712533995260403]
 [-0.03595888121020412 -0.0532051505082657 … 0.1533519972553758 0.1540201173328377; 0.0 -0.044231046046988395 … 0.15335056449113055 0.15391926272628703; … ; 0.0 -0.05070386237313112 … 0.10991025709206882 0.10995886357509516; -0.04987632842959719 -0.06217270868807964 … 0.10986118701690553 0.10991061230895155]
 [-0.17230807727459907 -0.30282876552957627 …

In [10]:
data=load("data/solpruba.jld","t")

20-element Vector{Float64}:
  0.49999999999999994
  1.0000000000000004
  1.5000000000000004
  2.0000000000000004
  2.4999999999999982
  2.9999999999999982
  3.4999999999999982
  3.9999999999999982
  4.499999999999998
  4.999999999999998
  5.499999999999998
  5.999999999999998
  6.499999999999998
  6.999999999999998
  7.499999999999998
  7.999999999999998
  8.500000000000007
  9.000000000000007
  9.500000000000007
 10.000000000000007

In [11]:
data.

LoadError: syntax: incomplete: premature end of input